# Beg Borrow and Steal: Snowplows

In [ ]:
import pandas as pd

In [ ]:
from bbs import snowplows

plows = snowplows.get_plow_data()
first = pd.Timestamp.fromtimestamp(plows["timestamp"].min() / 1000, tz="UTC")

In [ ]:
selected = snowplows.select_plow_data_for_timestamp(plows, first + pd.Timedelta("1h"))
selected

In [ ]:
df = selected.sort_values(["plow_id", "timestamp"]).copy()
df = pd.concat([df, df.groupby("plow_id").shift(1).add_prefix("previous_")], axis=1)
df["elapsed"] = (df["timestamp"] - df["previous_timestamp"]) / 1000
df = df.dropna()
df


In [ ]:
from matplotlib import pyplot as plt

for row in df.itertuples():
    plt.plot([row.previous_longitude, row.longitude], [row.previous_latitude, row.latitude], color="black", alpha=max(1 - (row.elapsed / (60 * 30)), 0.05))

In [ ]:
from matplotlib import animation
import numpy as np

def animate(df, first, last):
    fig, ax = plt.subplots()
    long_range = df.longitude.max() - df.longitude.min()
    xlim = [df.longitude.min() - long_range * 0.05, df.longitude.max() + long_range * 0.05]
    lat_range = df.latitude.max() - df.latitude.min()
    ylim = [df.latitude.min() - lat_range * 0.05, df.latitude.max() + lat_range * 0.05]

    def init():
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_title(f"Plow positions @ {first.tz_convert('US/Central')}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        fig.set_size_inches(8, 10)
        return ax.plot([], [], 'ro')
    
    def update(frame):
        ax.clear()
        target_time = frame
        ax.set_title(f"Plow positions @ {target_time.tz_convert('US/Central')}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        selected = snowplows.select_plow_data_for_timestamp(plows, target_time, interval=60*60*6)
        df = selected.sort_values(["plow_id", "timestamp"]).copy()
        df = pd.concat([df, df.groupby("plow_id").shift(1).add_prefix("previous_")], axis=1)
        df["elapsed"] = target_time.timestamp() - (df["timestamp"] / 1000)
        df = df.dropna()
        plots = []
        for row in df.itertuples():
            ln, = ax.plot([row.previous_longitude, row.longitude], [row.previous_latitude, row.latitude], color="black", alpha=max(1 - (row.elapsed / (60 * 30)), 0.05))
            plots.append(ln)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        return plots

    anim = animation.FuncAnimation(fig, update, frames=pd.date_range(first, last, freq="5min"), init_func=init, blit=True)
    return anim

In [ ]:
first = pd.Timestamp.fromtimestamp(plows["timestamp"].min() / 1000, tz="UTC")
last = pd.Timestamp.fromtimestamp(plows["timestamp"].max() / 1000, tz="UTC")
first = pd.Timestamp("2019-12-10T00:00:00Z")

dates = list(pd.date_range(first, last, freq="7d"))
it = iter(dates)
next(it)
dates = list(zip(dates, it))
for start, end in dates:
    anim = animate(plows, start, end)
    writervideo = animation.FFMpegWriter(fps=30, metadata=dict(artist='Kevin Schiroo'))
    anim.save(f"{start.date()}.mp4", writer=writervideo)